In [132]:
import pandas as pd
import reverse_geocoder as rg
import pycountry
import geopandas as gpd
from fuzzywuzzy import process

In [317]:
df = pd.read_csv('GTD.csv')

In [380]:
df =  pd.read_csv('small_arms_FINAL.csv',low_memory=False)
coords = pd.read_csv('country-coord.csv',low_memory=False)
coords = coords[['Country','Latitude (average)', 'Longitude (average)']]

In [394]:
df =  pd.read_csv('tradeRegister.csv',low_memory=False, encoding='ISO-8859-1')


In [395]:
#df2 = df[['Recipient', 'Supplier', 'Year(s) of delivery', 'SIPRI TIV of delivered weapons']]
df2 = df.dropna(subset=['Recipient', 'Supplier', 'Year(s) of delivery']).copy()

In [372]:
#df2 = df[['Reporter_Name', 'Partner_Name', 'Year', 'Value']]
df2 = df.dropna(subset=['Reporter_Name', 'Partner_Name', 'Year']).copy()


In [339]:
#df2 = df[['iyear', 'latitude', 'longitude', 'nkill' ]]
df2 = df.dropna(subset=['iyear', 'latitude', 'longitude']).copy()


In [85]:
def validate_country(name):
    try:
        return pycountry.countries.lookup(name).name
    except LookupError:
        return name

In [103]:
def get_country_code(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

In [373]:
df2['Partner_Name'] = df2['Partner_Name'].apply(validate_country)
df2['Reporter_Name'] = df2['Reporter_Name'].apply(validate_country)
#df2['Reporter_cc'] = df2['Reporter_Name'].apply(get_country_code)
#df2['Partner_cc'] = df2['Partner_Name'].apply(get_country_code)

In [370]:
df2['coordinates'] = list(zip(df2['latitude'], df2['longitude']))
coordinates = df2['coordinates']
results = rg.search(list(coordinates))
df2['cc'] = [results['cc']for results in results]

KeyError: 'latitude'

In [338]:
def get_cn(cc):
    try:
        if cc == 'XK':
            country_name = 'Kosovo'
        else:
            country_name = pycountry.countries.get(alpha_2=cc).name
    except:
        country_name = None
    return country_name


In [369]:
df2['country'] = df2['cc'].apply(get_cn)

KeyError: 'cc'

In [21]:
list1 = df2['country'].unique().tolist()

In [374]:
display(df2)

,Reporter_Code,Reporter_Name,Partner_Code,Partner_Name,ImportOrExport,Year,Period_Start,Period_End,Weapons_Type,Units,...,Weight,Licenses_Refused,AuthOrDel,GovtOrInd,Data_Source,Reliability,Accuracy,SmallArmsOnly,Comment,GlobalComment
0,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Kalashnikovs (all models),600,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
1,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Machine Guns,Not Known,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
2,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Mortars (unspecified calibre),Not Known,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
3,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Small Arms Ammunition,10000,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
4,700,Afghanistan,-2,All Countries,Import,1974,NaN,NaN,Non-Military Firearms (89431),2138780,...,NaN,NaN,Del,Not Known,© Comtrade,Sec/Hi,High,Yes,Partner country: Defined as World,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668779,552,Zimbabwe,551,Zambia,Import,2007,NaN,NaN,Sporting & Hunting Rifles (930330),1,...,NaN,NaN,Del,Not Known,© Comtrade,Sec/Hi,High,Yes,NaN,NaN
668780,552,Zimbabwe,551,Zambia,Import,2014,NaN,NaN,Sporting & Hunting Rifles (930330),1,...,NaN,NaN,Del,Not Known,© Comtrade,Sec/Hi,High,Yes,NaN,NaN
668781,552,Zimbabwe,551,Zambia,Import,2017,NaN,NaN,Sporting & Hunting Shotguns (930320),1,...,NaN,NaN,Del,Not Known,© Comtrade,Sec/Hi,High,Yes,NaN,NaN
668782,552,Zimbabwe,551,Zambia,Import,2018,NaN,NaN,Sporting & Hunting Shotguns (930320),1,...,NaN,NaN,Del,Not Known,© Comtrade,Sec/Hi,High,Yes,NaN,NaN


In [342]:
gtd = df2.copy()
gtd.drop(columns=['country_txt', 'cc', 'coordinates'], inplace=True)


In [343]:
gtd = gtd.dropna(subset=['country','iyear', 'latitude', 'longitude'])


In [345]:
columns_to_check = ['country', 'iyear', 'latitude', 'longitude']
na_presence = gtd[columns_to_check].isna().any()
print(na_presence)


country      False
iyear        False
latitude     False
longitude    False
dtype: bool


In [381]:
country_names = coords['Country'].unique()
unique_countries = pd.unique(df2[['Reporter_Name', 'Partner_Name']].values.ravel('K'))

In [356]:
def get_closest_name(query):
    best_match = process.extractOne(query, country_names)
    if best_match[1] >= 90:
        return best_match[0]
    else:
        return None 

country_match = {country: get_closest_name(country) for country in unique_countries}
keys_to_remove = [key for key, value in country_match.items() if key == value]

for key in keys_to_remove:
    del country_match[key]
keys_to_remove = [key for key, value in country_match.items() if key == value or value is None]

for key in keys_to_remove:
    del country_match[key]

In [361]:
display(df2.head())

,Reporter_Code,Reporter_Name,Partner_Code,Partner_Name,ImportOrExport,Year,Period_Start,Period_End,Weapons_Type,Units,...,Weight,Licenses_Refused,AuthOrDel,GovtOrInd,Data_Source,Reliability,Accuracy,SmallArmsOnly,Comment,GlobalComment
0,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Kalashnikovs (all models),600,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
1,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Machine Guns,Not Known,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
2,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Mortars (unspecified calibre),Not Known,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
3,700,Afghanistan,339,Albania,Import,2003,NaN,NaN,Small Arms Ammunition,10000,...,NaN,NaN,Del,Gov,Press,Sec/Hi,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN
4,700,Afghanistan,-2,All Countries,Import,1974,NaN,NaN,Non-Military Firearms (89431),2138780,...,NaN,NaN,Del,Not Known,© Comtrade,Sec/Hi,High,Yes,Partner country: Defined as World,NaN


In [359]:
display(country_names)

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia, Plurinational State of', 'Bosnia and Herzegovina',
       'Botswana', 'Bouvet Island', 'Brazil',
       'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands',
       'Colombia', 'Comoros', 'Congo',
       'Congo, the Democratic Republic of the', 'Cook Islands',
       'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic

In [366]:
valid_countries = set(country_names) | set(country_match.keys())
nd = [country for country in unique_countries if country not in valid_countries]
display(len(nd))

64

In [377]:
country_match2 = {
    'Antigua & Barbuda': 'Antigua and Barbuda',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Brunei': 'Brunei Darussalam',
    'Caribbean Netherlands (BES Islands) / Netherlands Antilles': 'Netherlands Antilles',
    'China (PRC)': 'China',
    'Congo (Republic of)': 'Congo',
    'Czechoslovakia': 'Slovakia',
    'Democratic Republic of the Congo / Zaire (former)': 'Congo, the Democratic Republic of the',
    'French Guyana': 'French Guiana',
    'Germany (Federal Republic)': 'Germany',
    'Libya': 'Libyan Arab Jamahiriya',
    'Macedonia': 'Macedonia, the former Yugoslav Republic of',
    'Micronesia (Federated States of)': 'Micronesia, Federated States of',
    'Niue Island': 'Niue',
    'Reunion': 'Réunion',
    'Samoa / Western Samoa (former) ': 'Samoa',
    'Sao Tome-Principe': 'Sao Tome and Principe',
    'Serbia / Yugoslavia (former) / Serbia & Montenegro (former)': 'Serbia',
    'St. Pierre and Miquelon': 'Saint Pierre and Miquelon',
    'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Surinam': 'Suriname',
    'Tonga Islands': 'Tonga',
    'Turks and Caicos': 'Turks and Caicos Islands',
    'Wallis and Futuna Islands': 'Wallis and Futuna',
    'Yemen Arab Republic (former)': 'Yemen',
    'Austria ': 'Austria',
    'Bulgaria ': 'Bulgaria',
    'Czech Republic ': 'Czech Republic',
    'Germany (Federal Republic) ': 'Germany',
    'Israel ': 'Israel',
    'Italy ': 'Italy',
    'Montenegro ': 'Montenegro',
    'Yugoslavia [former] / Serbia & Montenegro': 'Montenegro',
    'Belgium ': 'Belgium',
    'Brazil ': 'Brazil',
    'Finland ': 'Finland',
    'France ': 'France',
    'Spain ': 'Spain',
    'Switzerland ': 'Switzerland',
    'Turkey ': 'Turkey',
    'United States of America ': 'United States',
    'Portugal ': 'Portugal',
    'Serbia / Yugoslavia (former) / Serbia & Montenegro (former) ': 'Serbia',
    'Ukraine ': 'Ukraine',
    'China [PRC]': 'China',
    'Norfolk Islands': 'Norfolk Island',
    'Luxemburg': 'Luxembourg',
    'Belarus ': 'Belarus',
    'China (PRC) ': 'China',
    'Singapore ': 'Singapore',
    'Tokelau Islands': 'Tokelau',
    'Algeria ': 'Algeria',
    'Australia ': 'Australia',
    'Canada ': 'Canada',
    'Croatia ': 'Croatia',
    'Cyprus ': 'Cyprus',
    'Denmark ': 'Denmark',
    'Hong Kong ': 'Hong Kong',
    'Hungary ': 'Hungary',
    'Indonesia ': 'Indonesia',
    'Japan ': 'Japan',
    'Liechtenstein ': 'Liechtenstein',
    'Morocco ': 'Morocco',
    'Netherlands ': 'Netherlands',
    'New Zealand ': 'New Zealand',
    'Norway ': 'Norway',
    'Oman ': 'Oman',
    'Pakistan ': 'Pakistan',
    'Poland ': 'Poland',
    'Slovakia ': 'Slovakia',
    'Slovenia ': 'Slovenia',
    'South Africa ': 'South Africa',
    'Sweden ': 'Sweden',
    'United Kingdom ': 'United Kingdom',
    'Zaire / Democratic Republic of the Congo': 'Congo, the Democratic Republic of the',
    'Zambia ': 'Zambia',
    'British Virgin Islands ': 'Virgin Islands, British',
    'Greece ': 'Greece',
    'Iceland ': 'Iceland',
    'Lebanon ': 'Lebanon',
    'Macedonia ': 'Macedonia, the former Yugoslav Republic of',
    'Malaysia ': 'Malaysia',
    'Marshall Islands ': 'Marshall Islands',
    'Romania ': 'Romania',
    'Swaziland ': 'Swaziland',
    'Albania ': 'Albania',
    'Bosnia-Herzegovina ': 'Bosnia and Herzegovina',
    'India ': 'India',
    'Moldova ': 'Moldova, Republic of',
    'Serbia / Yugoslavia (former) / Serbia & Montenegro': 'Serbia',
    'Sri Lanka ': 'Sri Lanka',
    'Congo [Republic of]': 'Congo',
    'Christmas Islands': 'Christmas Island',
    'Jordan ': 'Jordan',
    'Africa': None,
    'Afghanistan ': 'Afghanistan',
    'Argentina ': 'Argentina',
    'Colombia ': 'Colombia',
    'Egypt ': 'Egypt',
    'Ethiopia ': 'Ethiopia',
    'Kazakhstan ': 'Kazakhstan',
    'Latvia ': 'Latvia',
    'Lithuania ': 'Lithuania',
    'Luxembourg ': 'Luxembourg',
    'Luxemburg ': 'Luxembourg',
    'Mexico ': 'Mexico',
    'Panama ': 'Panama',
    'Philippines ': 'Philippines',
    'Saudi Arabia ': 'Saudi Arabia',
    'Uruguay ': 'Uruguay',
    'Yemen Arab Republic [former]': 'Yemen',
    'Estonia ': 'Estonia',
    'Congo (Republic of; Brazzaville)': 'Congo',
    'Georgia ': 'Georgia',
    'Malta ': 'Malta',
    'Qatar ': 'Qatar',
    'Barbados ': 'Barbados',
    'Yugoslavia (former) / Serbia & Montenegro ': 'Montenegro',
    'Bahamas ': 'Bahamas',
    'Chile ': 'Chile',
    'Holy See': 'Holy See (Vatican City State)',
    'Sao Tome-Principe ': 'Sao Tome and Principe',
    'Guyana ': 'Guyana',
    'Puerto Rico ': 'Puerto Rico',
    'Nigeria ': 'Nigeria',
    'Falkland Islands': 'Falkland Islands (Malvinas)',
    'Nicaragua ': 'Nicaragua',
    'Thailand ': 'Thailand',
    'Mali ': 'Mali',
    'Namibia ': 'Namibia',
    'Surinam ': 'Suriname',
    'French Polynesia ': 'French Polynesia',
    'Kiribati ': 'Kiribati',
    'New Caledonia ': 'New Caledonia',
    'Norfolk Islands ': 'Norfolk Island',
    'United Arab Emirates ': 'United Arab Emirates',
    'Zimbabwe ': 'Zimbabwe',
    'Tanzania ': 'Tanzania, United Republic of',
    'Chad ': 'Chad',
    'Andorra ': 'Andorra',
    'Azerbaijan ': 'Azerbaijan',
    'Kyrgyzstan ': 'Kyrgyzstan',
    'Libya ': 'Libyan Arab Jamahiriya',
    'Ireland ': 'Ireland',
    'Virgin Islands': 'Virgin Islands, U.S.',
    'Western Samoa': 'Samoa',
    'Cocos [Keeling] Islands': 'Cocos (Keeling) Islands',
    'Trinidad and Tobago ': 'Trinidad and Tobago',
    'Heard Island and  McDonald  Islands': 'Heard Island and McDonald Islands',
    'Burma': 'Myanmar', 
    'Cameroun': 'Cameroon',
    'Czechia': 'Czech Republic',
    'East Timor': 'Timor-Leste',
    'German Democratic Republic (former)': 'Germany',
    'German Democratic Republic [former]': 'Germany',
    'Ivory Coast': "Côte d'Ivoire",
    'Korea; South (ROK)': 'Korea, Republic of',
    'Maldive Islands': 'Maldives',
    'Russia / USSR (Former)': 'Russian Federation', 
    'St. Kitts-Nevis': 'Saint Kitts and Nevis',
    'St. Lucia': 'Saint Lucia',
    'Taiwan (ROC) ': 'Taiwan, Province of China',
    'Vietnam; Republic of (former)': 'Viet Nam', 
    'West Bank': 'Palestinian Territory, Occupied',
    'All Countries': None,
    'Unspecified Country': None,
    'All Countries ': None,
    'Czechoslovalkia': 'Czech Republic',
    'German Democratic Republic \[former\]': 'Germany',
    'Europe': None,
    'Korea; North (DPRK)': "Korea, Democratic People's Republic of",
    'Americas': None,
    'Korea; South': 'Korea, Republic of',
    'Taiwan \[ROC\]': 'Taiwan, Province of China',
    'Taiwan [ROC]':'Taiwan, Province of China',
    'Russia / USSR (Former) ': 'Russian Federation',
    'Curacao': 'Netherlands',
    'Oceania': None, 
    'Korea; North': "Korea, Democratic People's Republic of",
    'Unspecified Country ': None,
    'Korea; South (ROK) ': 'Korea, Republic of',
    'St. Helena': 'Saint Helena, Ascension and Tristan da Cunha',
    'European Union ': None,
    'Taiwan (ROC)': 'Taiwan, Province of China',
    'U.S. Pacific Trust Territories': 'United States Minor Outlying Islands',
    'European Union': None,
    'Middle East ': None,
    'Non-EU Europe ': None,
    'North America ': None,
    'North East Asia ': None,
    'South America ': None,
    'South Asia ': None,
    'Sub- Saharan Africa ': None,
    'Central Asia ': None,
    'German Democratic Republic (former) ': 'Germany',
    'South East Asia ': None,
    'Sint Maarten': 'Anguilla',
    'St. Barthelemy': 'Anguilla',
    'Canary Islands': 'Spain',
    'Ceuta and Melilla': 'Spain',
    'NATO ': None,
    'Not Found': None,
    'Polar Regions': None,
    'Asia': None,
    'Carribean': None,
    'No Country ': None,
    'Korea; North (DPRK) ': "Korea, Democratic People's Republic of",
    'Bonaire': 'Venezuela, Bolivarian Republic of',
    'Curacao ': 'Venezuela, Bolivarian Republic of',
    'French Antarctic Territories': 'French Southern Territories',
    'Kosovo ': 'Kosovo',
    'St. Kitts-Nevis ': 'Saint Kitts and Nevis',
    'Ceuta': 'Spain',
    'Melilla': 'Spain',
    'Vietnam; Republic of \[former\]': 'Viet Nam',
    'Vietnam; Republic of [former]': 'Viet Nam'
}

In [ ]:
sat_country_match = {**country_match, **country_match2}
display(sat_country_match)

In [378]:
sat = df2.copy()
sat['Reporter_Name'] = sat['Reporter_Name'].map(country_match2).fillna(sat['Reporter_Name'])
sat['Partner_Name'] = sat['Partner_Name'].map(country_match2).fillna(sat['Partner_Name'])


In [392]:
countries = pd.unique(sat[['Reporter_Name', 'Partner_Name']].values.ravel('K'))
valid_countries = set(country_names) 
nd = [country for country in countries if country not in valid_countries]
display(countries)

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Myanmar',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Netherlands Antilles', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia',
       'Cuba', 'Cyprus', 'Czech Republic', 'Slovakia',
       'Congo, the Democratic Republic of the', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Timor-Leste', 'Ecuador',
       'Egypt', 'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia',
       'Faroe Island

In [ ]:
display(coords)

In [383]:
Sat = sat.copy()
Sat.rename(columns={'Reporter_Name': 'importer', 'Partner_Name': 'exporter'}, inplace=True)
Sat = Sat.merge(coords, how='left', left_on='importer', right_on='Country')
Sat.rename(columns={'Latitude (average)': 'latitude_importer', 'Longitude (average)': 'longitude_importer'}, inplace=True)
Sat.drop(columns=['Country'], inplace=True)  

Sat = Sat.merge(coords, how='left', left_on='exporter', right_on='Country')
Sat.rename(columns={'Latitude (average)': 'latitude_exporter', 'Longitude (average)': 'longitude_exporter'}, inplace=True)
Sat.drop(columns=['Country'], inplace=True)  
Sat = Sat.dropna(subset=['latitude_importer', 'longitude_importer', 'latitude_exporter', 'longitude_exporter'])

In [384]:
coordinates = list(zip(Sat['latitude_importer'], Sat['longitude_importer']))
results = rg.search(list(coordinates))
Sat['importer_cc'] = [results['cc']for results in results]
Sat = Sat.drop('importer', axis=1)


In [385]:
coordinates = list(zip(Sat['latitude_exporter'], Sat['longitude_exporter']))
results = rg.search(list(coordinates))
Sat['exporter_cc'] = [results['cc']for results in results]
Sat = Sat.drop('exporter', axis=1)

In [386]:
Sat['importer'] = Sat['importer_cc'].apply(get_cn)
Sat['exporter'] = Sat['exporter_cc'].apply(get_cn)
Sat = Sat.drop('importer_cc', axis=1)
Sat = Sat.drop('exporter_cc', axis=1)

In [393]:
allcountries = pd.unique(Sat[['importer', 'exporter']].values.ravel('K'))
display(allcountries)

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Myanmar',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cabo Verde',
       'Curaçao', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Slovakia',
       'Congo, The Democratic Republic of the', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Timor-Leste', 'Ecuador',
       'Egypt', 'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia',
       'Faroe Islands', 'Fiji', 'Finland', 'Fra

In [415]:
display(Sat)

,ImportOrExport,Year,Period_Start,Period_End,Weapons_Type,Units,Value,Currency,Licenses_Issued,Weight,...,Accuracy,SmallArmsOnly,Comment,GlobalComment,latitude_importer,longitude_importer,latitude_exporter,longitude_exporter,importer,exporter
0,Import,2003,NaN,NaN,Kalashnikovs (all models),600,NaN,NaN,NaN,NaN,...,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN,33.0,65.0,41.0000,20.0,Afghanistan,Albania
1,Import,2003,NaN,NaN,Machine Guns,Not Known,NaN,NaN,NaN,NaN,...,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN,33.0,65.0,41.0000,20.0,Afghanistan,Albania
2,Import,2003,NaN,NaN,Mortars (unspecified calibre),Not Known,NaN,NaN,NaN,NaN,...,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN,33.0,65.0,41.0000,20.0,Afghanistan,Albania
3,Import,2003,NaN,NaN,Small Arms Ammunition,10000,NaN,NaN,NaN,NaN,...,Med,Yes,2002. 'Albanian Defense Ministry Confirms Dona...,NaN,33.0,65.0,41.0000,20.0,Afghanistan,Albania
11,Import,1974,NaN,NaN,Non-Military Firearms (89431),50000,5553,USD,NaN,NaN,...,High,Yes,Partner country: Defined as Belgium-Luxembourg,NaN,33.0,65.0,50.8333,4.0,Afghanistan,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668779,Import,2007,NaN,NaN,Sporting & Hunting Rifles (930330),1,1779,USD,NaN,NaN,...,High,Yes,NaN,NaN,-20.0,30.0,-15.0000,30.0,Zimbabwe,Zambia
668780,Import,2014,NaN,NaN,Sporting & Hunting Rifles (930330),1,830,USD,NaN,NaN,...,High,Yes,NaN,NaN,-20.0,30.0,-15.0000,30.0,Zimbabwe,Zambia
668781,Import,2017,NaN,NaN,Sporting & Hunting Shotguns (930320),1,400,USD,NaN,NaN,...,High,Yes,NaN,NaN,-20.0,30.0,-15.0000,30.0,Zimbabwe,Zambia
668782,Import,2018,NaN,NaN,Sporting & Hunting Shotguns (930320),1,950,USD,NaN,NaN,...,High,Yes,NaN,NaN,-20.0,30.0,-15.0000,30.0,Zimbabwe,Zambia


In [414]:
Sat.drop(columns=['Reporter_Code', 'Partner_Code'], inplace=True)  

In [416]:
columns_to_check = ['importer', 'exporter', 'latitude_importer', 'longitude_importer', 'latitude_exporter', 'longitude_exporter','Year']
na_presence = Sat[columns_to_check].isna().any()
print(na_presence)


importer              False
exporter              False
latitude_importer     False
longitude_importer    False
latitude_exporter     False
longitude_exporter    False
Year                  False
dtype: bool


In [396]:
display(df2)

,Recipient,Supplier,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
0,Afghanistan,Belarus,2018.0,NaN,4.0,NaN,Mi-24V/Mi-35,combat helicopter,4.0,NaN,2019,Second hand,Second-hand; financed by India,3.40,13.60,13.60
1,Afghanistan,Bosnia-Herzegovina,2009.0,NaN,60.0,NaN,D-30 122mm,towed gun,60.0,?,2011,Second hand but modernized,Second-hand but modernized before delivery,0.23,13.86,13.86
2,Afghanistan,Bosnia-Herzegovina,2012.0,?,20.0,?,D-30 122mm,towed gun,20.0,NaN,2013,Second hand,Second-hand,0.14,2.80,2.80
3,Afghanistan,Brazil,2013.0,NaN,20.0,NaN,EMB-314 Super Tucano,trainer/combat aircraft,20.0,?,2016,New,$427 m 'LAS' deal (financed by USA); A-29B ver...,4.50,90.00,90.00
4,Afghanistan,Brazil,2017.0,NaN,6.0,NaN,EMB-314 Super Tucano,trainer/combat aircraft,6.0,?,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.00,27.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29034,Zimbabwe,Ukraine,2000.0,?,1.0,?,An-12,transport aircraft,1.0,?,2001,Second hand,Second-hand,9.20,9.20,9.20
29035,Zimbabwe,United States,1967.0,?,1.0,NaN,Baron,light aircraft,1.0,NaN,1967,New,For VIP transport,0.70,0.70,0.70
29036,Zimbabwe,United States,1993.0,?,2.0,NaN,Cessna-337/O-2,light aircraft,2.0,NaN,1993,Second hand,Second-hand; for anti-poaching operations,0.14,0.28,0.28
29037,Zimbabwe,Soviet Union,1975.0,?,15.0,?,T-34-85,tank,15.0,?,1975,Second hand,Second-hand; supplier uncertain,0.38,5.70,5.70


In [397]:
country_names = coords['Country'].unique()
unique_countries = pd.unique(df2[['Recipient', 'Supplier']].values.ravel('K'))

In [337]:
display(country_names)

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia, Plurinational State of', 'Bosnia and Herzegovina',
       'Botswana', 'Bouvet Island', 'Brazil',
       'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands',
       'Colombia', 'Comoros', 'Congo',
       'Congo, the Democratic Republic of the', 'Cook Islands',
       'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic

In [295]:
def get_closest_name(query):
    best_match = process.extractOne(query, country_names)
    if best_match[1] >= 90:
        return best_match[0]
    else:
        return None 

country_match = {country: get_closest_name(country) for country in unique_countries}
keys_to_remove = [key for key, value in country_match.items() if key == value]

for key in keys_to_remove:
    del country_match[key]
keys_to_remove = [key for key, value in country_match.items() if key == value or value is None]

for key in keys_to_remove:
    del country_match[key]

In [ ]:
display(country_match)

In [399]:
country_match ={
    'Bolivia': 'Bolivia, Plurinational State of',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Brunei': 'Brunei Darussalam',
    'Czechoslovakia': 'Slovakia',
    'DR Congo': 'Congo',
    'East Germany (GDR)': 'Germany',
    'Iran': 'Iran, Islamic Republic of',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Libya': 'Libyan Arab Jamahiriya',
    'Micronesia': 'Micronesia, Federated States of',
    'Moldova': 'Moldova, Republic of',
    'Northern Cyprus': 'Cyprus',
    'Russia': 'Russian Federation',
    'Saint Vincent': 'Saint Vincent and the Grenadines',
    'Syria': 'Syrian Arab Republic',
    'Taiwan': 'Taiwan, Province of China',
    'Tanzania': 'Tanzania, United Republic of',
    'Venezuela': 'Venezuela, Bolivarian Republic of',
    'FNLA (Angola)*': 'Angola',
    'Mujahedin (Afghanistan)*': 'Afghanistan',
    'Northern Alliance (Afghanistan)*': 'Afghanistan',
    'UNITA (Angola)*': 'Angola',
    'Anti-Castro rebels (Cuba)*': 'Cuba',
    'FAN (Chad)*': 'Chad',
    'GUNT (Chad)*': 'Chad',
    'Khmer Rouge (Cambodia)*': 'Cambodia',
    'ELF (Ethiopia)*': 'Ethiopia',
    'EPLF (Ethiopia)*': 'Ethiopia',
    'Viet Minh (France)*': 'Viet Nam',
    'Armas (Guatemala)*': 'Guatemala',
    'Haiti rebels*': 'Haiti',
    'PLO (Israel)*': 'Israel',
    'Indonesia rebels*': 'Indonesia',
    'Amal (Lebanon)*': 'Lebanon',
    'Hezbollah (Lebanon)*': 'Lebanon',
    'LF (Lebanon)*': 'Lebanon',
    'Lebanon Palestinian rebels*': 'Lebanon',
    'SLA (Lebanon)*': 'Lebanon',
    'United Wa State (Myanmar)*': 'Myanmar',
    'MTA (Myanmar)*': 'Myanmar',
    'Contras (Nicaragua)*': 'Nicaragua',
    'PRC (Israel/Palestine)*': 'Israel',
    'PIJ (Israel/Palestine)*': 'Israel',
    'MPLA (Portugal)*': 'Portugal',
    'PAIGC (Portugal)*': 'Portugal',
    'MNLF (Philippines)*': 'Philippines',
    'FRELIMO (Portugal)*': 'Portugal',
    'RPF (Rwanda)*': 'Rwanda',
    'ANC (South Africa)*': 'South Africa',
    'Darfur rebels (Sudan)*': 'Sudan',
    'FMLN (El Salvador)*': 'El Salvador',
    'UIC (Somalia)*': 'Somalia',
    'LTTE (Sri Lanka)*': 'Sri Lanka',
    'SPLA (Sudan)*': 'Sudan',
    'RUF (Sierra Leone)*': 'Sierra Leone',
    'SNA (Somalia)*': 'Somalia',
    'LRA (Uganda)*': 'Uganda',
    'Ukraine Rebels*': 'Ukraine',
    'Yemen Arab Republic (North Yemen)': 'Yemen',
    'Houthi rebels (Yemen)*': 'Yemen',
    'Southern rebels (Yemen)*': 'Yemen',
    'ZAPU (Zimbabwe)*': 'Zimbabwe',
    'North Yemen': 'Yemen',
    'South Yemen': 'Yemen',
    'African Union**': None,
    'Biafra': 'Nigeria',
    'Cabo Verde': 'Cape Verde',
    'Czechia': 'Czech Republic',
    'Katanga': 'Congo, the Democratic Republic of the',
    'North Korea': "Korea, Democratic People's Republic of",
    'South Korea': 'Korea, Republic of',
    'Kosovo': 'Kosovo',
    'Laos': "Lao People's Democratic Republic",
    'Libya HoR*': 'Libyan Arab Jamahiriya',
    'Libya GNC': 'Libyan Arab Jamahiriya',
    'North Macedonia': 'Macedonia, the former Yugoslav Republic of',
    'NATO**': None,
    'OSCE**': None,
    'Palestine': 'Palestinian Territory, Occupied',
    'Regional Security System**': None,
    'eSwatini': 'Swaziland',
    'Turkiye': 'Turkey',
    'UAE': 'United Arab Emirates',
    'United Nations**': None,
    'Soviet Union': 'Russian Federation',
    'South Vietnam': 'Viet Nam',
    'Kurdistan Regional Government (Iraq)*': 'Iraq',
    'NTC (Libya)*': 'Libyan Arab Jamahiriya',
    'Pathet Lao (Laos)*': "Lao People's Democratic Republic",
    'NLA (Macedonia)*': 'Macedonia, the former Yugoslav Republic of',
    'Hamas (Palestine)*': 'Palestinian Territory, Occupied',
    'Syria rebels*': 'Syrian Arab Republic',
    'PKK (Turkiye)*': 'Turkey',
    'Provisional IRA (UK)*': 'United Kingdom',
    'Viet Cong (South Vietnam)*': 'Viet Nam',
    'unknown recipient(s)': None,
    'unknown rebel group*': None,
    'Yugoslavia': 'Serbia',
    'unknown supplier(s)': None,
    'European Union**': None
    }
 

In [400]:
valid_countries = set(country_names) | set(country_match.keys())
nd = [country for country in unique_countries if country not in valid_countries]
display(nd)

[]

In [408]:
display(Lat)

,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,latitude_importer,longitude_importer,latitude_exporter,longitude_exporter,importer,exporter
0,2018.0,NaN,4.0,NaN,Mi-24V/Mi-35,combat helicopter,4.0,NaN,2019,Second hand,Second-hand; financed by India,3.40,13.60,13.60,33.0,65.0,53.0,28.0,Afghanistan,Belarus
3,2013.0,NaN,20.0,NaN,EMB-314 Super Tucano,trainer/combat aircraft,20.0,?,2016,New,$427 m 'LAS' deal (financed by USA); A-29B ver...,4.50,90.00,90.00,33.0,65.0,-10.0,-55.0,Afghanistan,Brazil
4,2017.0,NaN,6.0,NaN,EMB-314 Super Tucano,trainer/combat aircraft,6.0,?,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.00,27.00,33.0,65.0,-10.0,-55.0,Afghanistan,Brazil
5,2013.0,NaN,20.0,?,aircraft engine,aircraft engine,20.0,?,2016,New,Pratt & Whitney Canada PT-6A-68 (1250hp) turbo...,0.50,10.00,10.00,33.0,65.0,60.0,-95.0,Afghanistan,Canada
6,2017.0,NaN,6.0,?,aircraft engine,aircraft engine,6.0,?,2018,New,Pratt & Whitney Canada PT-6A-68 (1250hp) turbo...,0.50,3.00,3.00,33.0,65.0,60.0,-95.0,Afghanistan,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27944,2005.0,NaN,6.0,NaN,AI-25,turbofan,6.0,NaN,2005,New,For 6 K-8 trainer aircraft from China,0.60,3.60,3.60,-20.0,30.0,49.0,32.0,Zimbabwe,Ukraine
27945,2006.0,NaN,6.0,NaN,AI-25,turbofan,6.0,NaN,2006,New,For 6 K-8 trainer aircraft from China,0.60,3.60,3.60,-20.0,30.0,49.0,32.0,Zimbabwe,Ukraine
27946,2000.0,?,1.0,?,An-12,transport aircraft,1.0,?,2001,Second hand,Second-hand,9.20,9.20,9.20,-20.0,30.0,49.0,32.0,Zimbabwe,Ukraine
27947,1967.0,?,1.0,NaN,Baron,light aircraft,1.0,NaN,1967,New,For VIP transport,0.70,0.70,0.70,-20.0,30.0,38.0,-97.0,Zimbabwe,United States


In [402]:
Lat = df2.copy()
Lat.rename(columns={'Recipient': 'importer', 'Supplier': 'exporter'}, inplace=True)
Lat = Lat.merge(coords, how='left', left_on='importer', right_on='Country')
Lat.rename(columns={'Latitude (average)': 'latitude_importer', 'Longitude (average)': 'longitude_importer'}, inplace=True)
Lat.drop(columns=['Country'], inplace=True)  

Lat = Lat.merge(coords, how='left', left_on='exporter', right_on='Country')
Lat.rename(columns={'Latitude (average)': 'latitude_exporter', 'Longitude (average)': 'longitude_exporter'}, inplace=True)
Lat.drop(columns=['Country'], inplace=True)  
Lat = Lat.dropna(subset=['latitude_importer', 'longitude_importer', 'latitude_exporter', 'longitude_exporter'])

In [403]:
coordinates = list(zip(Lat['latitude_importer'], Lat['longitude_importer']))
results = rg.search(list(coordinates))
Lat['importer_cc'] = [results['cc']for results in results]
Lat = Lat.drop('importer', axis=1)

In [404]:
coordinates = list(zip(Lat['latitude_exporter'], Lat['longitude_exporter']))
results = rg.search(list(coordinates))
Lat['exporter_cc'] = [results['cc']for results in results]
Lat = Lat.drop('exporter', axis=1)

In [405]:
Lat['importer'] = Lat['importer_cc'].apply(get_cn)
Lat['exporter'] = Lat['exporter_cc'].apply(get_cn)
Lat = Lat.drop('importer_cc', axis=1)
Lat = Lat.drop('exporter_cc', axis=1)

In [311]:
Lat['year'] = Lat['Year(s) of delivery'].str.split(';').str[0].str.strip()
Lat.drop(columns=['Year(s) of delivery'], inplace=True)


In [407]:
allcountries = pd.unique(Lat[['importer', 'exporter']].values.ravel('K'))
display(allcountries)

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Austria',
       'Australia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados',
       'Bahamas', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bhutan',
       'Belarus', 'Belize', 'Botswana', 'Brazil', 'Bulgaria', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Cambodia', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
       'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'Equatorial Guinea',
       'Eritrea', 'Estonia', 'Timor-Leste', 'Ethiopia', 'Finland', 'Fiji',
       'France', 'Germany', 'Gabon', 'Gambia', 'Guinea-Bissau', 'Georgia',
       'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guyana', 'Haiti',
       'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland',
       'Iraq', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kiribati'

In [ ]:
years = Lat['year'].unique().tolist()
display(years)

In [410]:
Lat.to_csv('standardised_lat.csv', index=False, encoding='ISO-8859-1')
